In [2]:
#!python3.9 -m pip install Scrapy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.0 MB/s eta 0:00:0000:0100:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [1]:
from scrapy import Spider
from scrapy.selector import Selector
from scrapy.http import Request
from urllib.parse import urlencode

In [2]:
from scrapy.item import Item, Field

class mPage(Item):
	text = Field()

In [3]:
class mSpider(Spider):
    name = "crawl 1"
    allowed_domains = ['proxy.scrapeops.io', "fanfiction.net"]
    headers = {"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Mobile Safari/537.36",
        "accept": "application/json",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-US,en;q=0.9,en-GB;q=0.8,ar;q=0.7",
        "cache-control":" no-cache",
        "pragma": "no-cache",
        "referer": "https://steamdb.info/graph/", 
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest"}
    start_urls = [
        "https://www.fanfiction.net/tv/Sherlock/"
    ]

    API_KEY = ""
    def get_scrapeops_url(self, url):
        payload = {'api_key': self.API_KEY, 'url': url}
        proxy_url = 'https://proxy.scrapeops.io/v1/?' + urlencode(payload)
        return proxy_url

    def start_requests(self):
        for url in self.start_urls:
            proxy_url = self.get_scrapeops_url(url)
            yield Request(url=proxy_url, callback=self.parse, headers=self.headers)
            
    def parse(self, response):
        select = Selector(response)
        stories = select.css('a.stitle')
        for story in stories.xpath('@href').extract():
            url = 'https://www.fanfiction.net' + story
            proxy_url = self.get_scrapeops_url(url)
            yield Request(proxy_url, self.parseStoryPage, headers=self.headers)

        # index next page too
        next_page = select.xpath('//a[contains(text(), "Next")]/@href')
        print(len(stories))
        if len(next_page) > 0:
            next_page_url = 'https://www.fanfiction.net' + next_page[0].extract()
            proxy_url = self.get_scrapeops_url(next_page_url)
            yield Request(proxy_url, callback=self.parse, headers=self.headers)


    def parseStoryPage(self, response):
        select = Selector(response)
        textnodes = select.css('.storytext p').xpath('text()')
        language = ' '.join(select.css('#profile_top .xgray').xpath('text()').extract())
        print()
        if 'English' in language and len(textnodes) >= 1:
            print(response.url)
            front = response.url.split("%2F")[:-3]
            story_id = int(response.url.split("%2F")[-3])
            page_num = int(response.url.split("%2F")[-2])
            end = response.url.split("%2F")[-1:]

            text = textnodes.extract()
            page = mPage()
            page['text'] = u'\n'.join(text)
            with open('docs/' + str(story_id) + '.txt', 'a') as txtfile:
                unicode_text = page['text'] + u'\n'#).encode('utf8')
                txtfile.write(unicode_text)

            _page = '/'.join(front) + '/' + str(story_id) + '/' + str(page_num+1) + '/' + '/'.join(end)
            proxy_url = self.get_scrapeops_url(_page)
            # crawl the next page
            yield Request(url=proxy_url, callback=self.parseStoryPage, headers=self.headers)

In [4]:
from scrapy.crawler import CrawlerProcess

In [5]:
process = CrawlerProcess({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
})

process.crawl(mSpider)
process.start()

2024-12-04 10:36:49 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2024-12-04 10:36:49 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.11.0, Python 3.9.20 (main, Nov  6 2024, 15:08:59) - [Clang 12.0.0 (clang-1200.0.32.29)], pyOpenSSL 24.3.0 (OpenSSL 3.4.0 22 Oct 2024), cryptography 44.0.0, Platform macOS-10.15.7-x86_64-i386-64bit
2024-12-04 10:36:49 [scrapy.addons] INFO: Enabled addons:
[]
2024-12-04 10:36:49 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2024-12-04 10:36:49 [scrapy.extensions.telnet] INFO: Telnet Password: 5cc74312b262ee37
2024-12-04 10:36:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2024-12-04 10:36:49 [scrapy.crawler] INFO: Overridden settings:
{}
2024-12-04 10:36:50 [scr

25


2024-12-04 10:36:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F14409440%2F1%2FSlumber-in-Transit> (referer: https://steamdb.info/graph/)
2024-12-04 10:36:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F14286981%2F1%2FA-Man-From-All-Sides> (referer: https://steamdb.info/graph/)
2024-12-04 10:36:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F14411312%2F1%2FThrough-The-Fog-of-Illness> (referer: https://steamdb.info/graph/)



https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F14409440%2F1%2FSlumber-in-Transit

https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F14286981%2F1%2FA-Man-From-All-Sides


2024-12-04 10:36:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F14409571%2F1%2FHeat-of-the-Moment> (referer: https://steamdb.info/graph/)
2024-12-04 10:36:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F12343696%2F1%2FThere-I-Saw-You-Night> (referer: https://steamdb.info/graph/)


...

2024-12-04 10:42:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13782215%2F1%2FMycroft-s-Schooldays> (referer: https://steamdb.info/graph/)
2024-12-04 10:42:59 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13780914%2F1%2FNot-the-thought-that-counts> (failed 1 times): 429 Unknown Status



https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13782215%2F1%2FMycroft-s-Schooldays


2024-12-04 10:43:00 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13786282%2F1%2FShe-Came-In-Through-The-Bathroom-Window> (failed 1 times): 429 Unknown Status
2024-12-04 10:43:00 [scrapy.core.engine] DEBUG: Crawled (401) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13786600%2F1%2FIf-Convenient> (referer: https://steamdb.info/graph/)
2024-12-04 10:43:00 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <401 https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13786600%2F1%2FIf-Convenient>: HTTP status code is not handled or not allowed
2024-12-04 10:43:01 [scrapy.core.engine] DEBUG: Crawled (401) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13765198%2F1%2FThe-Christmas-Caper> (referer: https://steamdb.info/graph/)
2024-12-04 10:43:01 [scrapy.spidermiddlewares.httperror


https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13628784%2F1%2FThe-Adventures-of-Emma-Holmes


2024-12-04 10:43:05 [scrapy.core.engine] DEBUG: Crawled (401) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13788540%2F1%2FWe-Can-Work-It-Out> (referer: https://steamdb.info/graph/)
2024-12-04 10:43:05 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <401 https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13788540%2F1%2FWe-Can-Work-It-Out>: HTTP status code is not handled or not allowed
2024-12-04 10:43:06 [scrapy.core.engine] DEBUG: Crawled (401) <GET https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13788556%2F1%2FAn-Unexpected-Gift> (referer: https://steamdb.info/graph/)
2024-12-04 10:43:06 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <401 https://proxy.scrapeops.io/v1/?api_key=&url=https%3A%2F%2Fwww.fanfiction.net%2Fs%2F13788556%2F1%2FAn-Unexpected-Gift>: HTTP status code is not handled or not allowed
2024-12-04 10:43:06 [scrapy.core.engine] DEBUG: 